# 挂载数据

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# 导入库

In [0]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf

# 加载数据

In [0]:
def load_data(root, start, end):
    all_files = glob.glob(os.path.join(root, '*.npy'))
    
    x = np.empty([0, 784])
    y = np.empty([0])
    class_names = []
    
    
    for index, file in enumerate(all_files, start=0):
        data = np.load(file)
        data = data[start:end, :]
        labels = np.full(data.shape[0], index)
        
        x = np.concatenate((x, data), axis=0)
        y = np.append(y, labels)
        class_name, ext = os.path.splitext(os.path.basename(file))
        class_names.append(class_name)
        
    permutation = np.random.permutation(y.shape[0])
    x = x[permutation, :]
    y = y[permutation]
    
    return x, y, class_names
    

# 预处理数据

In [0]:
def preprocess_data(x, y):
    image_size = 28
    num_classes = len(class_names)

    x = x.reshape(x.shape[0], image_size, image_size, 1).astype('float32')
    x /= 255.0
    y = keras.utils.to_categorical(y, num_classes)
    return x, y

# 构建模型

In [5]:
# 构建模型
model = keras.Sequential()

# 三组卷积层和池化层
# 参数input_shape为(28, 28, 1)
model.add(keras.layers.Conv2D(16, (3, 3), padding='same', input_shape=(28, 28, 1), activation=tf.nn.relu))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(32, (3, 3), padding='same', activation=tf.nn.relu))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), padding='same', activation=tf.nn.relu))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))

# 展平
model.add(keras.layers.Flatten())

# 全连接
model.add(keras.layers.Dense(512, activation=tf.nn.relu))
model.add(keras.layers.Dense(345, activation=tf.nn.softmax))

model.summary()

# 编译
model.compile(optimizer=tf.train.AdamOptimizer(), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flat

# 训练模型

In [6]:
num = 1000 # 由于内存有限，每次训练1000组数据
for i in range(10):
    #加载数据
    x, y, class_names = load_data('/content/drive/My Drive/mini-quickdraw-dataset/', start=num*i, end=num*(i+1))
    #预处理数据
    x, y = preprocess_data(x, y)
    print("已加载{}组数据".format(num*(i+1)))
    #拟合模型
    model.fit(x=x, y=y, batch_size=256, epochs=10, verbose=1, validation_split=0.1)
    print("已完成{}组数据训练".format(num*(i+1)))

已加载1000组数据
Train on 310500 samples, validate on 34500 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
310500/310500 [==============================] - 21s 66us/sample - loss: 3.1567 - acc: 0.3247 - val_loss: 2.4032 - val_acc: 0.4514
Epoch 2/10
310500/310500 [==============================] - 18s 57us/sample - loss: 2.1836 - acc: 0.4940 - val_loss: 2.0853 - val_acc: 0.5181
Epoch 3/10
310500/310500 [==============================] - 18s 56us/sample - loss: 1.9272 - acc: 0.5457 - val_loss: 1.9609 - val_acc: 0.5410
Epoch 4/10
310500/310500 [==============================] - 18s 56us/sample - loss: 1.7799 - acc: 0.5752 - val_loss: 1.9101 - val_acc: 0.5532
Epoch 5/10
310500/310500 [==============================] - 17s 55us/sample - loss: 1.6785 - acc: 0.5962 - val_loss: 1.8663 - val_acc: 0.5628
Epoch 6/10
310500/310500 [==============================] - 18s 57us/sample - loss: 1.5988 - acc: 0.6115 - val_loss: 1.8531 - val_acc: 0.5687
Epoch 7/10
310500/310500 [============

# 保存模型

In [8]:
model.save('/content/keras.h5')